# 教育销售聊天机器人

## 构造销售话术数据

Prompt：

```
你是中国顶级的中小学课程销售，现在培训职场新人。请给出100条使用的销售话术。内容包括但不限于：中小学课程、中考和高考准备、出国留学择校、大学考试专业选择、艺术类考试准备。

每条销售话术以如下格式给出（保留每行开头的“[客户问题]”和“[销售回答]”）：
<序号>
[客户问题]
[销售回答]
```

## 使用 Document Transformers 模块来处理原始数据

In [4]:
with open("education_sales_data.txt") as f:
    education_sales = f.read()

In [5]:
# 使用 CharacterTextSplitter 来进行文本分割

from langchain.text_splitter import CharacterTextSplitter

In [8]:
text_splitter = CharacterTextSplitter(        
    separator = '\n\n',
    chunk_size = 200,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)

In [9]:
docs = text_splitter.create_documents([education_sales])

In [10]:
docs[0]

Document(page_content='[客户问题] 我在考虑是否为我的孩子报读中小学课程，有什么建议吗？\n[销售回答] 当然，我们的中小学课程设计注重学生的全面发展，包括学科知识和兴趣培养。您可以考虑为孩子选择一门有趣的特长，如音乐、体育或者艺术，这样可以帮助他们更好地发展个人潜能。', metadata={})

In [11]:
len(docs)

55

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [14]:
query = "孩子读书压力大，怎么办？"

In [15]:
answer_list = db.similarity_search(query)

In [16]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我觉得孩子的学习压力太大了，怎么办？
[销售回答] 学习压力是孩子学习过程中的常见问题。我们可以与您的孩子进行深入的交流和沟通，了解他们的学习压力来源，并提供一些有效的减压方法和建议。同时，我们还可以提供一些心理辅导和支持，帮助他们更好地应对学习压力和挑战。

[客户问题] 我的孩子学习效率不高，怎么办？
[销售回答] 学习效率是学习成果的重要因素之一。我们可以为您的孩子提供一些有效的学习方法和技巧，如快速阅读、笔记整理、时间管理等，帮助他们提高学习效率。

[客户问题] 我觉得孩子在学校里没有得到足够的关注和重视，怎么办？
[销售回答] 学校教育应该关注每个孩子的个性和需求。您可以与孩子的老师进行积极的沟通和合作，共同关注孩子的需求和发展。此外，我们还可以为您的孩子提供个性化的辅导和支持，帮助他们更好地融入学校生活和学习。

[客户问题] 我的孩子想尝试发表文章或出版自己的书籍，有什么推荐吗？
[销售回答] 发表文章或出版自己的书籍是展示孩子才华的好方式。我们可以为您的孩子提供一些发表文章或出版书籍的建议和资源，如写作指导、编辑服务、出版资源等，帮助他们更好地实现自己的创作梦想。



In [17]:
db.save_local("education_sales")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [18]:
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [19]:
topK_retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7fe5539b9570>, search_type='similarity', search_kwargs={'k': 3})

In [20]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我觉得孩子的学习压力太大了，怎么办？
[销售回答] 学习压力是孩子学习过程中的常见问题。我们可以与您的孩子进行深入的交流和沟通，了解他们的学习压力来源，并提供一些有效的减压方法和建议。同时，我们还可以提供一些心理辅导和支持，帮助他们更好地应对学习压力和挑战。

[客户问题] 我的孩子学习效率不高，怎么办？
[销售回答] 学习效率是学习成果的重要因素之一。我们可以为您的孩子提供一些有效的学习方法和技巧，如快速阅读、笔记整理、时间管理等，帮助他们提高学习效率。

[客户问题] 我觉得孩子在学校里没有得到足够的关注和重视，怎么办？
[销售回答] 学校教育应该关注每个孩子的个性和需求。您可以与孩子的老师进行积极的沟通和合作，共同关注孩子的需求和发展。此外，我们还可以为您的孩子提供个性化的辅导和支持，帮助他们更好地融入学校生活和学习。



In [21]:
docs = topK_retriever.get_relevant_documents("有没有中学数学的课程？")

In [22]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我的孩子需要提高数学应用能力，有什么针对性课程推荐吗？
[销售回答] 针对数学应用能力，我们可以为您的孩子提供一些针对性课程，如数学建模、统计学、离散数学等，让他们在数学应用领域有更深入的学习和研究。

[客户问题] 我的孩子需要提高奥数成绩，有什么课程或资源推荐吗？
[销售回答] 奥数是数学领域中的一项重要挑战。我们可以为您的孩子提供一些奥数课程和资源，如华罗庚数学、奥林匹克数学等，让他们在奥数领域有更深入的学习和研究。

[客户问题] 我在考虑是否为我的孩子报读中小学课程，有什么建议吗？
[销售回答] 当然，我们的中小学课程设计注重学生的全面发展，包括学科知识和兴趣培养。您可以考虑为孩子选择一门有趣的特长，如音乐、体育或者艺术，这样可以帮助他们更好地发展个人潜能。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [23]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [24]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我觉得孩子的学习压力太大了，怎么办？
[销售回答] 学习压力是孩子学习过程中的常见问题。我们可以与您的孩子进行深入的交流和沟通，了解他们的学习压力来源，并提供一些有效的减压方法和建议。同时，我们还可以提供一些心理辅导和支持，帮助他们更好地应对学习压力和挑战。

[客户问题] 我的孩子学习效率不高，怎么办？
[销售回答] 学习效率是学习成果的重要因素之一。我们可以为您的孩子提供一些有效的学习方法和技巧，如快速阅读、笔记整理、时间管理等，帮助他们提高学习效率。

[客户问题] 我觉得孩子在学校里没有得到足够的关注和重视，怎么办？
[销售回答] 学校教育应该关注每个孩子的个性和需求。您可以与孩子的老师进行积极的沟通和合作，共同关注孩子的需求和发展。此外，我们还可以为您的孩子提供个性化的辅导和支持，帮助他们更好地融入学校生活和学习。

[客户问题] 我的孩子想尝试发表文章或出版自己的书籍，有什么推荐吗？
[销售回答] 发表文章或出版自己的书籍是展示孩子才华的好方式。我们可以为您的孩子提供一些发表文章或出版书籍的建议和资源，如写作指导、编辑服务、出版资源等，帮助他们更好地实现自己的创作梦想。



### 提取向量数据库中的`销售回答`

In [25]:
docs = retriever.get_relevant_documents(query)

In [26]:
docs[0].page_content

'[客户问题] 我觉得孩子的学习压力太大了，怎么办？\n[销售回答] 学习压力是孩子学习过程中的常见问题。我们可以与您的孩子进行深入的交流和沟通，了解他们的学习压力来源，并提供一些有效的减压方法和建议。同时，我们还可以提供一些心理辅导和支持，帮助他们更好地应对学习压力和挑战。'

In [27]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我觉得孩子的学习压力太大了，怎么办？\n',
 '学习压力是孩子学习过程中的常见问题。我们可以与您的孩子进行深入的交流和沟通，了解他们的学习压力来源，并提供一些有效的减压方法和建议。同时，我们还可以提供一些心理辅导和支持，帮助他们更好地应对学习压力和挑战。']

In [28]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [29]:
ans

'学习压力是孩子学习过程中的常见问题。我们可以与您的孩子进行深入的交流和沟通，了解他们的学习压力来源，并提供一些有效的减压方法和建议。同时，我们还可以提供一些心理辅导和支持，帮助他们更好地应对学习压力和挑战。'

#### 尝试各种问题

In [30]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [32]:
query = "我想考警校"

print(sales(query))

[]


In [35]:
print(sales(query, 0.5))

['除了学校教育，我们还提供各种拓展课程和活动，如领导力培训、社会实践、科技创新等，旨在帮助您的孩子拓展视野、培养综合素质和增强社会竞争力。', '我们的中考和高考准备课程旨在提高学生的应试能力。我们的教师拥有丰富的教学经验和专业的背景，他们将帮助学生掌握考试技巧，提供精准的复习计划，助力学生取得好成绩。', '参加交换生项目可以让孩子体验不同的文化和教育体系。我们与多家国内外优质教育机构合作，可以为您提供丰富的交换生项目选择。您可以根据孩子的需求和兴趣为他们推荐最适合的交换生项目。', '竞赛类比赛是检验孩子学科能力和综合素质的好机会。我们可以为您的孩子提供专业的竞赛培训和指导，帮助他们提高竞赛能力和水平。此外，我们还可以协助他们准备比赛所需的材料和作品，让他们更有信心参加比赛。']


In [43]:
query = "课程费用2万以内的课程有哪些？"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: []

score:0.75 ans: []

score:0.5 ans: ['当然，我们可以为您的孩子提供一系列的扩展课程和深入研究的机会，包括参加专题讲座、参加学术研讨、组织研究项目等，让他们在感兴趣的领域有更深入的学习和研究。', '当然，我们可以为您的孩子提供一系列的扩展课程和研究项目，让他们在感兴趣的领域有更深入的学习和研究。此外，我们还可以为您推荐一些优质的学习资源和实践平台。', '针对经济学科，我们可以为您的孩子提供一些扩展课程和资源，如微观经济学、宏观经济学、国际经济等方面的内容。此外，我们还可以推荐一些经济类图书和网络资源，帮助他们更深入地了解经济世界。', '为孩子选择一个合适的留学学校是一个重要的决定。我们可以为您提供专业的留学咨询服务，根据孩子的需求和兴趣为他们推荐最合适的学校。此外，我们还可以提供国际教育交流机会，让孩子更好地了解不同国家的文化和教育体系。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [37]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [45]:
qa_chain({"query": query})

{'query': '课程费用2万以内的课程有哪些？',
 'result': '很抱歉，我无法提供具体的课程名称和费用范围。课程费用会因地区、学校、课程类型和学习方式等因素而有所不同。建议您通过学校或在线教育平台查询相关课程的费用信息。'}

In [40]:
qa_chain({"query": "我想考警校"})

{'query': '我想考警校', 'result': '很好！考警校是一个很好的选择。你有什么关于考警校的问题吗？'}

In [46]:
print(sales("我想考警校"))

[]


## 加载 FAISS 向量数据库已有结果

In [47]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("./education_sales/", OpenAIEmbeddings())

In [48]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [49]:
qa_chain({"query": "我想买别墅，你们有么"})

/home/ubuntu/.local/lib/python3.10/site-packages/langchain/vectorstores/base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供房地产销售服务。建议您联系当地的房地产中介或者房产开发商，他们可以帮助您找到合适的别墅。'}

In [33]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [34]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供房地产销售服务。建议您联系当地的房地产中介或者通过房地产网站来寻找别墅的购买信息。'}

In [35]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [36]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [37]:
result

{'query': '我想买别墅，你们有么',
 'result': '很抱歉，我是一个AI助手，无法提供实际的房产销售信息。建议您联系当地的房地产中介或者在房产网站上搜索别墅的销售信息。',
 'source_documents': []}